### Import libraries

In [8]:
import pandas as pd
import holidays
from datetime import timedelta, time

### Set priority by solution time

In [9]:
def sla_expected(priority: str) -> int:
    if priority == "High":
        return 24
    elif priority == "Medium":
        return 72
    elif priority == "Low":
        return 120
    else:
        return None

### Validation only on business days.

In [11]:
def is_business_day(current_date: pd.Timestamp, holiday_set: set) -> bool:
    if current_date.weekday() >= 5:
        return False
    if current_date.date() in holiday_set:
        return False
    return True

def gen_holiday_set(start_date: pd.Timestamp, end_date: pd.Timestamp, country: str = "BR") -> set:
    years = range(start_date.year, end_date.year + 1)
    holiday_calendar = holidays.CountryHoliday(country, years=years)
    return set(holiday_calendar.keys())

### Calculate working hours based on creation date and end date.

In [12]:
def calc_business_hrs(created_at: str, resolved_at: str, business_start_hour: int = 8, business_end_hour: int = 18) -> float:
    start_dt = pd.to_datetime(created_at, utc=True)
    end_dt = pd.to_datetime(resolved_at, utc=True)

    if pd.isna(start_dt) or pd.isna(end_dt):
        return 0.0
    if end_dt <= start_dt:
        return 0.0

    holiday_set = gen_holiday_set(start_dt, end_dt)
    total_hours = 0.0
    current_day = start_dt.normalize()

    while current_day <= end_dt.normalize():
        if is_business_day(current_day, holiday_set):
            business_start = pd.Timestamp.combine(
                current_day.date(),
                time(business_start_hour, 0),
            ).tz_localize("UTC")
            business_end = pd.Timestamp.combine(
                current_day.date(),
                time(business_end_hour, 0),
            ).tz_localize("UTC")

            interval_start = max(start_dt, business_start)
            interval_end = min(end_dt, business_end)

            if interval_end > interval_start:
                diff = interval_end - interval_start
                total_hours += diff.total_seconds() / 3600
        current_day += timedelta(days=1)
    return round(total_hours, 2)

### Assess compliance with the SLA.

In [13]:
def check_sla_compliance(created_at: str, resolved_at: str, priority: str) -> dict:
    resolution_hours = calc_business_hrs(
        created_at=created_at,
        resolved_at=resolved_at
    )

    sla_expected_hours = sla_expected(priority)

    if sla_expected_hours is None:
        is_sla_met = None
    else:
        is_sla_met = resolution_hours <= sla_expected_hours
    return {
        "resolution_hours": resolution_hours,
        "sla_expected_hours": sla_expected_hours,
        "is_sla_met": is_sla_met
    }